# **Building a CNN**


Estimated time needed: **30** minutes


## A mission to automate monitoring of flowering

Climate warning is causing a lot of changes in the timing and duration of flowering seasons, making it hard for the flower planting company to monitor the growing of various species. To study the changes, the company's biologists are monitoring the plants in small permanently marked areas and performing manual collection and analysis of the plant phenology details. 

You are now hired by the company as a Data Scientist to help automate the monitoring process. The first step of your mission is to create a flower type identification system, so that it could greatly reduce the time and cost of tracking the flowers on an indivifual level.

<center><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/images/flowershop.jpeg" width="80%"></center>


In this lab, we will tie together everything we've learned in previous CNN labs and apply our new found skills in a object recognition task.

This lab takes concepts learned in this Deep Learning and Reinforcement Learning course and applies them in the creation of a CNN model. 

We will be implementing a model that classifies images employing multiple convolutional **filters** for multi-**channel**/RGB images, adding **padding** to images to preserve image sizes/capture edge data, determining the best **stride** to use with the convolutional filters, passing that data through **activation functions** such as ReLU and **pooling** layers, and **flattenning** our results to obtain classes using **categorical cross entropy**.


## Table of Contents

<ol>
    <li><a href="https://#Objectives">Objectives</a></li>
    <li>
        <a href="https://#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>     
    </li>
    <li>
        <a href="#Example: Classifying Flowers">Example: Classifying Flowers</a>
        <ol>
            <li><a href="#Importing data">Importing data</a></li>
            <li><a href="#Building a classifier">Building a classifier</a></li>
            <li><a href="#Prediction!">Prediction!</a></li>
        </ol>   
    </li>
</ol>


## Objectives

After completing this lab you will be able to:

*   Explain how a convolution works on images
*   Understand the purposes of different kernels that exist
*   Apply kernels to images and obtain a useful result


***


## Setup


For this lab, we will be using the following libraries:

*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`Pillow`](https://pillow.readthedocs.io/en/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for image processing functions.
*   [`OpenCV`](https://docs.opencv.org/4.x/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for other image processing functions.
*   [`tensorflow`](https://www.tensorflow.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and neural network related functions.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Anaconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the code cell below.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy numpy==1.22.3 matplotlib==3.5.1 tensorflow==2.9.0 opencv-python==4.5.5.62

# Note: If your environment doesn't support "!mamba install", use "!pip install --user"

# RESTART YOUR KERNEL AFTERWARD AS WELL

### Importing Required Libraries

*We recommend you import all required libraries in one place (here):*


In [ ]:
import warnings
warnings.simplefilter('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import pathlib
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import PIL
from PIL import Image, ImageOps
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Defining Helper Functions


This function helps visualize the feature maps of the layers in a classifier.


In [ ]:
def plot_activations_multilayer(num_layers, images_per_row, classifier, activations):
    layer_names = []
    for layer in classifier.layers[:num_layers]:
        layer_names.append(layer.name + ' layer')  # Names of the layers, so you can have them as part of your plot
    for layer_name, layer_activation in zip(layer_names, activations):  # Displays the feature maps
        n_features = layer_activation.shape[-1]  # Number of features in the feature map
        size = layer_activation.shape[1]  # The feature map has shape (1, size, size, n_features).
        n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols): # Tiles each filter into a big horizontal grid
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :,
                                                 col * images_per_row + row]
                display_grid[col * size : (col + 1) * size, # Displays the grid
                             row * size : (row + 1) * size] = channel_image
        scale = 2. / size
        plt.figure(figsize=(scale*display_grid.shape[1],
                            scale*display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

## Example: Classifying Flowers


We will use our flower images from the image convolution lab. However, this time, we want to use them to train a CNN model for classification. Hence, we will utilize a training and test set.
https://www.tensorflow.org/tutorials/load_data/images


### Importing data

Lets take a look at the flowers dataset from tensorflow, retrieved from here: [https://www.tensorflow.org/datasets/catalog/tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01)


In [ ]:
import skillsnetwork

dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/datasets/flower_photos.tgz"
await skillsnetwork.prepare(dataset_url, overwrite=True)

In [ ]:
data_dir = Path("flower_photos")

for folder in data_dir.glob('[!LICENSE]*'):
    print('The', folder.name, 'folder has',
          len(list(folder.glob('*.jpg'))), 'pictures')

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count, 'total images')

Let's extract some images we can use for this lab. We will set them all to be square images of 300x300, and display them as well.


In [ ]:
img_width = 150
img_height = 150

batch_size = 64
epochs = 10

In [ ]:
# Plotting out some images we have
pics = list()
pics_arr = list()
p_class = list()

plt.figure(figsize=(20,5))
for idx, folder in enumerate(data_dir.glob('[!LICENSE]*')):
    cat = list(data_dir.glob(folder.name + '/*'))
    pic = PIL.Image.open(str(cat[0])).resize((img_width, img_height))
    pic_arr = np.array(pic)
    clss = folder.name
    
    plt.subplot(1,5,idx+1)
    plt.imshow(pic)
    plt.title(clss)
    plt.axis('off')
    
    pics.append(pic)
    pics_arr.append(pic_arr)
    p_class.append(clss)
    

Let's create a train set using the `ImageDataGenerator` and `flow_from_directory` functions from `keras.utils`.


In [ ]:
train_gen = ImageDataGenerator(validation_split=0.2, 
                               rescale=1.0/255.0,
                                width_shift_range=0.2, # 0.1
                                height_shift_range=0.2, # 0.1
                                horizontal_flip=True)
train_set = train_gen.flow_from_directory(
                               directory=data_dir,
                               seed=10,
                               class_mode='sparse',
                               batch_size=batch_size,
                               shuffle=True,
                               target_size=(img_height, img_width),
                               subset='training')

#### Exercise: Create validation set


In [ ]:
## Create a validation set using the same way, labelling subset as "validation"
# WRITE YOUR CODE HERE


<details><summary>Solution</summary>
    <code>val_gen = ImageDataGenerator(validation_split=0.2, 
                                rescale=1.0/255.0,
                                width_shift_range=0.2, 
                                height_shift_range=0.2,
                                horizontal_flip=True)
val_set = val_gen.flow_from_directory(
                               directory=data_dir,
                               seed=10,
                               class_mode='sparse',
                               batch_size=batch_size,
                               shuffle=True,
                               target_size=(img_height, img_width),
                               subset='validation')</code>
</details>


This creates a dictionary which we can use to look up the name of the flower type according to its label number.


In [ ]:
class_names = {y: x for x, y in val_set.class_indices.items()}
class_names

### Building a classifier


Define the model and rescale RGB image pixels to take on values between 0-1:


In [ ]:
classifier = Sequential()

#### Exercise: Define the first set of convolutional layers

Add the following layers to our classifier:
1. Convolutional layer with input depth equal to 3, 32 5x5 filters, even padding, and relu activation function.
2. Max pooling layer with size 2x2.
3. Convolutional layer with 64 3x3 filters, padding, and relu activation function.
3. Max pooling layer with size 2x2, strides of 2 horizontally and vertically.


In [ ]:
# WRITE YOUR CODE HERE


<details><summary>Solution</summary>
<code>
classifier.add(Conv2D(32, (5, 5), padding='same', input_shape = (img_width, img_height, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
</code></br>
</details>


#### Exercise: Add a second set of convolutional layers

Add the following layers to our classifier:
1. Convolutional layer with 32 3x3 filters, even padding, and relu activation function.
2. Max pooling layer with size 2x2, strides of 2 horizontally and vertically.
3. Convolutional layer with 32 3x3 filters, padding, and relu activation function.
3. Max pooling layer with size 2x2, strides of 2 horizontally and vertically.


In [ ]:
# WRITE YOUR CODE HERE


<details><summary>Solution</summary>
    <code>classifier.add(Conv2D(32, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
classifier.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))</code>
</details>


Let's look at the summary of our CNN construction:


In [ ]:
classifier.build((1,img_width, img_height,3))
classifier.summary()

Let's try to see how the layers look so far when applied on a sample image without training (call the `predict` method directly on the **img_tensor**.


In [ ]:
# display the sample image
img_tensor = np.array(pics_arr[2], dtype='int')
plt.imshow(img_tensor)

Before we input the image to the CNN, we have to add the batch dimension using ```np.expand_dims```


In [ ]:
img_tensor = np.expand_dims(img_tensor, axis=0)
y = classifier.predict(img_tensor)
print(f"The predicted output of the sample image has a shape of {y.shape}.")

From the summary above we saw there are 7 layers in **classifier**. We can use the helper function **plot_activations_multilayer** to visualize the feature maps produced by each layer before training.


In [ ]:
layer_outputs = [layer.output for layer in classifier.layers] 
activation_model = Model(inputs=classifier.input, outputs=layer_outputs) 
activations = activation_model.predict(img_tensor)

plot_activations_multilayer(8, 8, classifier, activations)

Although the **classifier** model hasn't been trained, it is already evident that certain features are getting recognized in each separate layer. Now, let's proceed with building the model for classification.


#### Exercise: Add a Flattening layer


From the summary above, we see that the shape of the output of the previous layer is $73\times73\times6$. Thus, the shape of the output of our flattening layer will be  $73\times73\times6 = 31974\times1$.


In [ ]:
# WRITE YOUR CODE HERE


<details><summary>Solution</summary>
    <code>classifier.add(Flatten()) </code></br>
</details>


#### Exercise: Add Dense Layers


Now the last step of building the **classifier** model is adding some fully-connected dense layers:

- Dense layer with 512 units and relu activation function.
- Dense layer with 5 units (because the dataset has 5 classes) and softmax activation function.


In [ ]:
# WRITE YOUR CODE HERE


<details><summary>Solution</summary>
    <code>classifier.add(Dense(units = 512, activation = 'relu'))</code></br>
    <code>classifier.add(Dense(units = 5, activation = 'softmax'))</code></br>
</details>


In [ ]:
classifier.summary()

### Prediction!

We compile the model using the Adam optimizer, categorical cross entropy as the loss function, and measuring performance based on its accuracy.


In [ ]:
classifier.compile(optimizer='adam', 
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

Let's fit the classifier on the train set and evaluate its performance against the validation set 

**Note: This may take some time, go make yourself a coffee while waiting!**


In [ ]:
classifier.fit(
  train_set,
  validation_data=val_set,
  epochs=epochs
)

Now that the network is trained and the kernels (the weight matrices of the layers) are learnt, we can visualize the kernels. For each Conv2d layer in the list of `classifier.layers`, it consists of multiple kernels (filters). To match the depth of the input to a Conv2d layer, each kernel's depth will have the same value as the input depth. The kernel depth is also referred to as the number of channels of a kernel. 

For example, if the input to the first Conv2d layer is a 3-channel RGB image, then each kernel of the layer will consist of three channels (depth 3) where each channel can be visualized as a 2D grayscale image. Depending on how many different feature maps we want a Conv2d layer to learn, we specify the number of kernels (filters) in a layer. 

With this idea in mind, let's look at how many kernels are learnt by our classifier: 


In [ ]:
for layer in classifier.layers:
    if 'conv2d' in layer.name:
        kernels, biases = layer.get_weights()
        print(f"layer name: {layer.name}, num of kernels: {kernels.shape[-1]}, kernel shape: {kernels.shape[:2]}, kernel depth: {kernels.shape[2]}")

You can see that except for the first Conv2d layer, others all have depth of 32 or 64, which means to visualize each kernel we would need to plot out 32 or 64 channels of it! To not bombard you with hundreads and thousands of kernel images, we will just visualize 3 channels of 4 kernels from each Conv2d layer:


In [ ]:
for layer in classifier.layers:
    if 'conv2d' in layer.name:
        name = layer.name

        kernels, _ = layer.get_weights()
        k_min, k_max = kernels.min(), kernels.max()
        kernels = (kernels - k_min) / (k_max - k_min)

        for i in range(4):
            kernel = kernels[:,:,:,i]
            fig = plt.figure(figsize=(5, 2))
            fig.suptitle(f"{name}, kernel {i+1}", fontsize=15)

            for j in range(3):
                plt.subplot(1, 3, j+1)
                plt.imshow(kernel[:,:,j], cmap='gray')
                plt.xticks([])
                plt.yticks([])


The dark squares indicate small or inhibitory weights of the kernel and the light squares represent large or excitatory weights of the kernel. Using this intuition, we can see that, a kernel (filter) like the following detects a gradient from light in the top left to dark in the bottom right.


In [ ]:
plt.imshow(kernels[:,:,2,3], cmap='gray')

After seeing the kernels used for convolution, let's take a look again at what the feature maps produced by the trained, intermediate layers look like:


In [ ]:
layer_outputs = [layer.output for layer in classifier.layers]
activation_model = Model(inputs=classifier.input, outputs=layer_outputs)

In [ ]:
# let's pick a sample image 
img_tensor = pics_arr[1]
plt.imshow(np.array(img_tensor, dtype='int'))

In [ ]:
img_tensor = np.expand_dims(img_tensor, axis=0)

activations = activation_model.predict(img_tensor)[:6]

plot_activations_multilayer(7,8,classifier,activations)

Compared to before, it's clear that after fitting the model on the train set, each consecutive layer is able to learn some abstract features. Let's predict the label for the sample image **img_tensor**.


In [ ]:
y = classifier.predict(img_tensor)
label = class_names[np.argmax(y)]

plt.imshow(img_tensor.reshape((img_width,img_height,3)).astype("uint8"))
plt.title(f"Predicted class is: {label}", fontsize=13)